# E: Web Scraping

In [3]:
import os
from datetime import datetime
import pandas as pd
import requests

## Part 1: Manually scraping sz.de

In [4]:
url = "http://sz.de"

In [5]:
response = requests.get(url, allow_redirects=True)

In [6]:
response.status_code

200

In [7]:
print(response.text[0:700], "[...]")

<!doctype html>
<html lang="de">

<head>

    <meta charset="utf-8" />
    <title>Aktuelle Nachrichten, Hintergründe und Kommentare - SZ.de</title>
    <link rel="canonical" href="https://www.sueddeutsche.de" />
    <meta name="robots" content="index,follow,noarchive,noodp" />
    <meta name="referrer" content="origin-when-cross-origin" />
    <meta name="author" content="Süddeutsche.de GmbH, Munich, Germany" />
    <meta name="copyright" content="Süddeutsche.de GmbH, Munich, Germany" />
    <meta name="email" content="kontakt@sueddeutsche.de" />
    <meta name="description" content="News aus Deutschland und aller Welt mit Kommentaren und Hintergrundberichten auf Süddeutsche.de." />
    <met [...]


In [8]:
print(response.content[0:700], "[...]")

b'<!doctype html>\n<html lang="de">\n\n<head>\n\n    <meta charset="utf-8" />\n    <title>Aktuelle Nachrichten, Hintergr\xc3\xbcnde und Kommentare - SZ.de</title>\n    <link rel="canonical" href="https://www.sueddeutsche.de" />\n    <meta name="robots" content="index,follow,noarchive,noodp" />\n    <meta name="referrer" content="origin-when-cross-origin" />\n    <meta name="author" content="S\xc3\xbcddeutsche.de GmbH, Munich, Germany" />\n    <meta name="copyright" content="S\xc3\xbcddeutsche.de GmbH, Munich, Germany" />\n    <meta name="email" content="kontakt@sueddeutsche.de" />\n    <meta name="description" content="News aus Deutschland und aller Welt mit Kommentaren und Hintergrundberichten auf S\xc3\xbcddeutsche.de." />\n    ' [...]


In [9]:
response.encoding

'UTF-8'

In [10]:
response.url

'https://www.sueddeutsche.de/'

In [11]:
response.elapsed

datetime.timedelta(microseconds=143191)

In [12]:
with open(os.path.join("local", "sz.html"), "wb") as f:
    f.write(response.content)

FileNotFoundError: [Errno 2] No such file or directory: 'local\\sz.html'

## Part 2: automatically scraping multiple websites

In [11]:
STORAGE_DIR = "local"

In [12]:
# List of news pages to be scraped
newspaper_urls = dict(
    sz="https://www.sueddeutsche.de/",
    zeit="https://www.zeit.de/index",
    faz="https://www.faz.net/aktuell/",
    ts="https://www.tagesspiegel.de/",
    spiegel="https://www.spiegel.de/",
    kronen="https://www.krone.at/",
    wtf="https://asdfkajwlkejwkejklajsdflksadjfasdf.nix",
)

In [13]:
# Current date as string
now = datetime.now()
now_str = now.strftime("%Y-%m-%d")
print(now_str)

2021-03-22


## Target objects

We will create two objects:

- `content_dict`: a dict with the HTML content of the pages we scraped
- `log_list`: a list with metadata about our requests

In [14]:
content_dict = {}
text_dict = {}
log_list = []
failing_list = []

## Scraper function

In [15]:
def scrape_website(name, url):
    
    # (1) Run request
    response = requests.get(url, allow_redirects=True)
    content = response.content
    text = response.text
    
    # (2) File name to store the raw HTML
    file_name = os.path.join(
        STORAGE_DIR,
        f"{now_str}-{name}.html",
    )
    
    # (3) Write raw HTML
    with open(file_name, "wb") as f:
        f.write(response.content)
        
    # (4) Fill content_dict and text_dict
    content_dict[name] = response.content
    text_dict[name] = response.text
    
    # (5) Fill log_list
    log_info = dict(
        name=name,
        date=now_str,
        file_name=file_name,
        status=response.status_code,
        url=response.url,
        encoding=response.encoding,
    )
    log_list.append(log_info)

## Execute

In [16]:
for name, url in newspaper_urls.items():
    try:
        scrape_website(name, url)
    except:
        failing_list.append((name, url))

## Check

In [17]:
log_df = pd.DataFrame(log_list)
log_df

,name,date,file_name,status,url,encoding
0,sz,2021-03-22,local\2021-03-22-sz.html,200,https://www.sueddeutsche.de/,UTF-8
1,zeit,2021-03-22,local\2021-03-22-zeit.html,200,https://www.zeit.de/index,UTF-8
2,faz,2021-03-22,local\2021-03-22-faz.html,200,https://www.faz.net/aktuell/,utf-8
3,ts,2021-03-22,local\2021-03-22-ts.html,200,https://www.tagesspiegel.de/,utf-8
4,spiegel,2021-03-22,local\2021-03-22-spiegel.html,200,https://www.spiegel.de/,utf-8
5,kronen,2021-03-22,local\2021-03-22-kronen.html,200,https://www.krone.at/,ISO-8859-1


In [18]:
log_file_name = os.path.join(
    STORAGE_DIR,
    f"{now_str}.csv",
)

In [19]:
log_df.to_csv(log_file_name)

In [20]:
failing_list

[('wtf', 'https://asdfkajwlkejwkejklajsdflksadjfasdf.nix')]